# Naive Bayes on Political Text
## Gonzalo Blazquez

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [27]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

from string import punctuation

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns
from nltk.corpus import stopwords
punctuation = set(punctuation) # speeds up comparison

# Stopwords
sw = set(stopwords.words("english"))

def remove_stop(tokens) :
    tokens2 = [word for word in tokens if word not in sw]
    return(tokens2)
 
def remove_punctuation(text, punct_set=punctuation) : 
    return(''.join([ch for ch in text if ch not in punct_set]))

def tokenize(text) :     
    tokens = text.split()
    return(tokens)

def lower_tokens(tokens):
    return [token.lower() for token in tokens]

def join_tokens(tokens):
    return ' '.join(tokens)

def isalpha_tokens(tokens):
    return [token for token in tokens if token.isalpha()]


In [5]:
# Get data from the conventions data base file
convention_db = sqlite3.connect("/users/Gonzalo B/Downloads/AppliedTextMining/Module4/Assign4/data/2020_Conventions.db")
convention_cur = convention_db.cursor()

In [6]:
tables = convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
tables
columns = convention_cur.execute(f"PRAGMA table_info({'conventions'});").fetchall()
columns

[(0, 'party', 'TEXT', 0, None, 0),
 (1, 'night', 'INTEGER', 0, None, 0),
 (2, 'speaker', 'TEXT', 0, None, 0),
 (3, 'speaker_count', 'INTEGER', 0, None, 0),
 (4, 'time', 'TEXT', 0, None, 0),
 (5, 'text', 'TEXT', 0, None, 0),
 (6, 'text_len', 'TEXT', 0, None, 0),
 (7, 'file', 'TEXT', 0, None, 0)]

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [7]:
convention_data = []

# fill the above list up with items that are themselves lists. The 
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text as speech, party as party
                            FROM conventions
                            WHERE party != 'Other';
                            ''')

for row in query_results :
    # store the results in convention_data
    speech, party = row
    convention_data.append([speech, party])

In [8]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right. 

In [9]:
random.choices(convention_data,k=5)

[['That means we don’t have to be afraid of you at all, right.', 'Republican'],
 ['And there is so much a new president can do to bring us together.',
  'Democratic'],
 ['When factory workers in Ohio are faced with dangerous conditions because this administration hasn’t given clear guidance on how to protect our people.',
  'Democratic'],
 ['Our party is united in offering you a very different choice, a go-to-work president. A down-to-earth, get-the-job-done guy. A man with a mission to take responsibility, not shift the blame; concentrate, not distract; unite, not divide. Our choice is Joe Biden.',
  'Democratic'],
 ['When I first got elected I came to Washington DC to fight for the issues that are so important in our community.',
  'Democratic']]

It'll be useful for us to have a large sample size than 2024 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html). 

In [ ]:
# 2024 instructions
#conv_sent_data = []

#for speech, party in convention_data :
 #   pass # your code here

Again, let's look at some random entries. 

In [ ]:
#random.choices(conv_sent_data,k=5)

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps: 

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [72]:
clean_conv_sent_data = [] # list of tuples (speech, party), with sentence cleaned

for idx, (speech, party) in enumerate(convention_data):
    
    speech_punct = remove_punctuation(speech, punctuation) # before token becuase it works with text
    tokens = tokenize(speech_punct)
    tokens = isalpha_tokens(tokens)
    tokens = lower_tokens(tokens) # before sw to remove 'The'
    tokens = remove_stop(tokens)
    cleaned_sentence = join_tokens(tokens)
    
    # store the results
    clean_conv_sent_data.append((cleaned_sentence, party))

random.choices(clean_conv_sent_data,k=5)

[('speaking strength determination recently celebrated hundred year anniversary passage amendment yesterday north lawn white house unveiled exhibit dedicated suffrage exhibit called children across country send art honoring meaning important time history judging entries reflected impact voices nation story proud cast vote donald november must make sure women heard american dream continues thrive',
  'Republican'),
 ('best campaign going international', 'Republican'),
 ('seventh grade english teacher mrs b used tell us none hear half read witness meaning words never fully weighed met husband trump family preconceived notion family disappeared immediately warm caring hard workers earth reminded family made feel like home walking halls trump organization saw family environment also saw countless women executives thrived year year gender matter mattered ability get job done learned directly fatherinlaw asked help win cherished home state namesake north carolina though political experience 

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5. 

In [101]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2255 as features in the model.


With a word cutoff of 5, we have 2255 as features in the model.

With a word cutoff of 10, we have 1352 as features in the model.

With a word cutoff of 20, we have 754 as features in the model.

With a word cutoff of 30, we have 500 as features in the model.

In [64]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    ret_dict = dict()
    words=tokenize(text)

    for word in words:
        if word in fw:
            ret_dict[word] = True 
    
    return(ret_dict)

In [102]:
assert(len(feature_words)>0)
assert(conv_features("obama was the president",feature_words)==
       {'obama':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [103]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [104]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [105]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.498


In [ ]:
10 0.482   5 0.498      30 0.474    20 0.468  1 0.492

In [106]:
classifier.show_most_informative_features(25)

Most Informative Features
             enforcement = True           Republ : Democr =     27.5 : 1.0
                   votes = True           Democr : Republ =     21.6 : 1.0
                 climate = True           Democr : Republ =     17.3 : 1.0
                 destroy = True           Republ : Democr =     17.1 : 1.0
                supports = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.9 : 1.0
                preserve = True           Republ : Democr =     15.1 : 1.0
                  signed = True           Republ : Democr =     15.1 : 1.0
              appreciate = True           Republ : Democr =     14.0 : 1.0
                freedoms = True           Republ : Democr =     14.0 : 1.0
                 private = True           Republ : Democr =     11.9 : 1.0
                  defund = True           Republ : Democr =     10.9 : 1.0
                    drug = True           Republ : Democr =     10.3 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [ ]:
cong_db = sqlite3.connect("congressional_twitter_data.db")
cong_cur = cong_db.cursor()

In [ ]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [ ]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [ ]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [ ]:

for tweet, party in tweet_data_sample :
    estimated_party = '??' # you need to fill this in.
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [ ]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [ ]:
results

### Reflections

_Write a little about what you see in the results_ 